## Airbnb Price Prediction - Optimizing Listings
#### Patrick Huston & Filippos Lymperopoulos | Spring 2016

*This notebook aims to document our process in creating a ML pipeline to predict Airbnb listing prices given an input set of features. A major focus of this exploration is to write modular, well-designed components that could easily be taken and applied to a different modeling situation.*

## TODO:

0. What should we do with space?
1. Explore encoding for bed_type
2. Explore encoding for room_type
3. Explore encoding for property_type
4. Explore encoding for amenities
5. Explore log-loss + understand accuracy meaning (what is .28? is that good?)
6. Document our code and decisions we've made
    - CleanProcessor: filling nulls, binary encoding, representing features
    - Explain our decisions in feature engineering and why we did them
    - Modeling decisions
7. Nulls in room_type? Get them from the space description
8. Is there anything interesting in the name of the listing?


### NOTES

1. Mean absolute deviation
2. Log transform the price - how far away from the order of magnitude

In [17]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import pickle
from sklearn import preprocessing
import seaborn as sns
from datetime import datetime
import binaryHelper as be
import sumHelper as se
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.externals import joblib

from sklearn import linear_model
from sklearn import svm

### The Data

Airbnb provides an expansive listings dataset for public use. It includes all listings for major cities around the world. For the purpose of our exploration, we'll start off by using well-known US cities - Boston, San Francisco, Los Angeles, Washington DC, and Seattle. For each listing, Airbnb provides a large amount of features - check [here](https://github.com/flymperopoulos/DataScience16CTW/blob/master/report/listings_features.md) to see them all listed out.

#### Load in the Listings Data

In [2]:
listingsBoston = pd.read_csv('../data/listingsBoston.csv')
listingsSF = pd.read_csv('../data/listingsSF.csv')
listingsLA = pd.read_csv('../data/listingsLA.csv')
listingsDC = pd.read_csv('../data/listingsDC.csv')
listingsSeattle = pd.read_csv('../data/listingsSeattle.csv')

frames = [listingsBoston, listingsSF, listingsLA, listingsDC, listingsSeattle]

listingsAll = pd.concat(frames)

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Load in the Calendar Data

In [3]:
calendarBoston = pd.read_csv('../data/calendarBoston.csv')
calendarSF = pd.read_csv('../data/calendarSF.csv')
calendarLA = pd.read_csv('../data/calendarLA.csv')
calendarDC = pd.read_csv('../data/calendarDC.csv')
calendarSeattle = pd.read_csv('../data/calendarSeattle.csv')

frames = [calendarBoston, calendarSF, calendarLA, calendarDC, calendarSeattle]

calendarAll = pd.concat(frames)

### Cleaning Convenience

To facilitate the process of cleaning, we've defined a cleaning helper and cleaning processor class below. 

In [48]:
def create_ordinal_mapping(data, parameter):
    parameter_mapping = {}
    
    for index, parameter in enumerate(sorted(data[parameter].unique())):
        parameter_mapping[parameter] = index
    
    return parameter_mapping

neighbourhood_mapping = create_ordinal_mapping(listingsAll, "neighbourhood_cleansed")
bed_type_mapping = create_ordinal_mapping(listingsAll, "bed_type")
room_type_mapping = create_ordinal_mapping(listingsAll, "room_type")
property_type_mapping = create_ordinal_mapping(listingsAll, "property_type")

# Sorting based on room quality
shared_room_ord = room_type_mapping["Shared room"] 
entire_apt_ord = room_type_mapping["Entire home/apt"]
room_type_mapping["Shared room"] = entire_apt_ord
room_type_mapping["Entire home/apt"] = shared_room_ord

print property_mapping

{nan: 0, 'Tipi': 21, 'Lighthouse': 15, 'Apartment': 1, 'Earth House': 11, 'Other': 17, 'Cabin': 5, 'Tent': 20, 'Chalet': 8, 'Loft': 16, 'Yurt': 26, 'Train': 23, 'Treehouse': 24, 'Hut': 13, 'Plane': 19, 'Dorm': 10, 'Camper/RV': 6, 'Bed & Breakfast': 2, 'Island': 14, 'Bungalow': 4, 'Castle': 7, 'Parking Space': 18, 'Villa': 25, 'House': 12, 'Condominium': 9, 'Townhouse': 22, 'Boat': 3}


#### Helper

The class `Helper` exposes a set of helpful functions that each deal with processing an individual feature in the dataset. For example, `amenities_to_list` takes in an individual row from the `amenities` column - represented as a string - and converts it into a more useful list format. 

In [49]:
class Helper():
    def __init__(self, data_attrs):
        self.data_attrs = data_attrs
    
    # Converts string representation of amenities list 
    def amenities_to_list(self, amenities):
        amenities = amenities.replace('{', '')
        amenities = amenities.replace('}', '')
        amenities = amenities.replace('\"', '')
        return amenities.split(',')

    # Creates new feature out of the number of amenities
    def num_amenities(self, amenitiesList):
        return len(amenitiesList)

    # Converts string representation of price to float 
    def price_to_int(self, price):
        price = price.replace(',', '')
        price = price.replace('$', '')
        return float(price)
    
    def date_to_days(self, startDay):
        dStart = datetime.strptime(startDay, "%Y-%m-%d")
        dEnd = datetime.strptime(self.data_attrs['date_min'], "%Y-%m-%d")
        return abs((dEnd - dStart).days)
    
    def row_to_ordinal(self, row, mapping):
        return self.data_attrs[mapping][row]

    
helper = Helper({'date_min': listingsAll[listingsAll.host_since.isnull() == False].host_since.max(), 
                 'neighbourhood_mapping': neighbourhood_mapping, 
                 'bed_type_mapping':bed_type_mapping, 
                 'room_type_mapping':room_type_mapping,
                 'property_type_mapping':property_type_mapping
                });

#### Clean Processor
The class `cleanProcessor` does handles two things - it applies the methods defined in `Helper` to its dataset and performs some additional processing like null-filling.

In [85]:
class cleanProcessor():
    def __init__(self, data):
        self.data = data
        
    def clean_listings(self):
        df_clean = self.data.copy()
        
        # TODO: What? - Better techniques for filling nulls
        df_clean.loc[df_clean.review_scores_rating.isnull(), 'review_scores_rating'] = 90
        df_clean.loc[df_clean.host_since.isnull(), 'host_since'] = '2015-10-02'
        df_clean.loc[df_clean.bedrooms.isnull(), 'bedrooms'] = 0
        df_clean.loc[df_clean.bathrooms.isnull(), 'bathrooms'] = 0
        df_clean.loc[df_clean.beds.isnull(), 'beds'] = 0
        df_clean.loc[df_clean.property_type.isnull(), 'property_type'] = "Other"
        
        df_clean['amenities'] = df_clean['amenities'].apply(helper.amenities_to_list)
        df_clean['num_amenities'] = df_clean['amenities'].apply(helper.num_amenities)
        df_clean['price'] = df_clean['price'].apply(helper.price_to_int)
        df_clean['days_host'] = df_clean['host_since'].apply(helper.date_to_days)
        df_clean['neighbourhood_binary'] = df_clean['neighbourhood_cleansed'].apply(helper.row_to_ordinal, args=("neighbourhood_mapping",))
        df_clean['property_sum_encoded'] = df_clean['property_type'].apply(helper.row_to_ordinal, args=("property_type_mapping",))
        
        # Binary encoding for neighborhoods
        encoder = be.BinaryEncoder(cols=['neighbourhood_binary'])
        binary_neighbourhoods = encoder.transform(df_clean)
        
        # Sum encoding for property type
        sum_encoder = be.BinaryEncoder(cols=['property_sum_encoded'])
        sum_properties = sum_encoder.transform(df_clean)
        
        # One-hot and Ordinal Encoding for bed_type and room_type
        bed_type = pd.get_dummies(df_clean.bed_type)    
        df_clean["bed_type"] = df_clean["bed_type"].apply(helper.row_to_ordinal, args=("bed_type_mapping",))
        room_type = pd.get_dummies(df_clean.room_type)    
        df_clean["room_type"] = df_clean["room_type"].apply(helper.row_to_ordinal, args=("room_type_mapping",))

        # One-hot encoding for cancellation policy
        cancellation_policy = pd.get_dummies(df_clean.cancellation_policy)
        cancellation_policy.rename(columns={'strict':'cancellation_strict'})
        cancellation_policy.rename(columns={'flexible':'cancellation_flexible'})
        cancellation_policy.rename(columns={'moderate':'cancellation_moderate'})
                
        data = pd.concat([df_clean, cancellation_policy, binary_neighbourhoods, bed_type, room_type], axis=1)

        return data

    def clean_calendar(self):
        self.df_clean = df.copy()
        self.df_clean = self.df_clean[self.df_clean.available == 't']
        self.df_clean['price'] = self.df_clean['price'].apply(helper.price_to_int)
        return self.df_clean

clean_processor_listings = cleanProcessor(listingsAll)
clean_processor_calendar = cleanProcessor(calendarAll)

In [86]:
listingsClean = clean_processor_listings.clean_listings()

In [8]:
print '{} -- {} '.format('Private', listingsClean[listingsClean.room_type == 1].price.median()) 
print '{} -- {} '.format('Shared', listingsClean[listingsClean.room_type == 0].price.median()) 
print '{} -- {} '.format('Entire', listingsClean[listingsClean.room_type == 2].price.median()) 

Private -- 80.0 
Shared -- 47.0 
Entire -- 159.0 


In [9]:
print bed_type_mapping

{'Real Bed': 4, 'Futon': 2, 'Couch': 1, 'Pull-out Sofa': 3, 'Airbed': 0}


In [10]:
print '{} -- {} '.format('Real', listingsClean[listingsClean.bed_type == 4].price.median())
print '{} -- {} '.format('Pull-out', listingsClean[listingsClean.bed_type == 3].price.median()) 
print '{} -- {} '.format('Futon', listingsClean[listingsClean.bed_type == 2].price.median()) 
print '{} -- {} '.format('Air', listingsClean[listingsClean.bed_type == 0].price.median()) 
print '{} -- {} '.format('Couch', listingsClean[listingsClean.bed_type == 1].price.median()) 

Real -- 120.0 
Pull-out -- 79.5 
Futon -- 70.0 
Air -- 65.0 
Couch -- 55.0 


### Choosing Models

Now that we've done a good amount of initial preprocessing on our data, let's move towards some predictive modeling. Our goal is to develop a model that will predict a price given listing parameters. We'll experiment with including different features, and see which combination fo features and models produces the best results.

After some initial research we've decided to start off by trying four different models - 

1. Linear Lasso
       The Lasso is a linear model that estimates sparse coefficients. It is useful in some contexts due to its tendency to prefer solutions with fewer parameter values, effectively reducing the number of variables upon which the given solution is dependent.
       
2. ElasticNet 
       ElasticNet is a linear regression model trained with L1 and L2 prior as regularizer. This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge.
       
3. Support Vector Regression
       Support Vector Regression is an implementation of regression that uses the SVM approach. In this case, we'll be using the linear kernel.
            
4. Ridge Regression
       Ridge regression addresses some of the problems of Ordinary Least Squares by imposing a penalty on the size of coefficients. The ridge coefficients minimize a penalized residual sum of squares.
       



In [87]:
# Linear Lasso
lasso = linear_model.Lasso(alpha = 0.1)

# ElasticNet
elasticNet = linear_model.ElasticNet(alpha = 0.1, l1_ratio=0.7)

# Ridge Regression
ridgeRegression = linear_model.Ridge(alpha = .5)

# Support Vector Regression
svr = svm.SVR(C=1.0, epsilon=0.2)

models = {'lasso': lasso, 'elasticNet': elasticNet, 'ridgeRegression': ridgeRegression }

### Testing Models

Now, let's define `ModelHelper`, a class that will facilitate the process of testing our models.

In [88]:
class ModelHelper():
    ''' 
    ModelHelper exposes a set of functions aimed at facilitating the dataset
    manipulation (train-test splitting) and model testing process
    '''
    
    def __init__(self, X, y, features):
        self.X = X
        self.y = y
        self.XFeat = X[features]
    
    def cross_validate(self, model, cv=3):
        '''Cross-validates model within trainnig set with a split of 'cv' - default value of 3'''
        cv = StratifiedKFold(self.X.neighbourhood_cleansed)
        return cross_val_score(model, self.XFeat, self.y, cv=cv).mean()

    def train_test_splitter(self, model, train_size=0.6, save=False):
        '''Performs train-test split on data, trains on train, tests on test, returns score, model, data'''
        X_train, X_test, y_train, y_test = train_test_split(self.XFeat, self.y, train_size=train_size)
        model.fit(X_train, y_train)
        return X_train, X_test, y_train, y_test, model

    def test_models(self, models):
        '''Iterates over all different models and print out their results of train_test_splitter'''
        for modelName, model in models.iteritems():
            print '{} : {}'.format(modelName, self.cross_validate(model))
    
    def save_model(self, fitted_model, filename="model.pkl"):
        '''Persists model to disk for later use in backend API'''
        joblib.dump(fitted_model, filename) 

In [89]:
sum_encoded_list= []
for i in range(26):
    sum_encoded_list.append("property_sum_encoded_{}".format(i))

In [90]:
# TODO: add property_type to features --> encoding
features = ['days_host', 'num_amenities', 'bedrooms', 'beds',
            'neighbourhood_binary_0', 'neighbourhood_binary_1', 
            'neighbourhood_binary_2', 'neighbourhood_binary_3',
            'neighbourhood_binary_4', 'neighbourhood_binary_5',
            'neighbourhood_binary_6', 'neighbourhood_binary_7',
            'neighbourhood_binary_8', 'bathrooms', 'accommodates',
            'bed_type', 'Airbed', 'Couch', 'Futon', 'Pull-out Sofa',
            'Real Bed', 'room_type', 'Entire home/apt', 'Private room',
            'Shared room']

allFeatures = features + sum_encoded_list


pickle.dump(allFeatures, open( '../app/utils/modelFeatures.pkl', 'wb' ))

In [91]:
mHelper = ModelHelper(listingsClean, listingsClean.price, features)

mHelper.test_models(models)

elasticNet : 0.273435882183
ridgeRegression : 0.273776643271
lasso : 0.273765463073


In [16]:
listingsClean.property_type.unique()

array(['House', 'Apartment', 'Villa', 'Condominium', 'Other',
       'Bed & Breakfast', 'Loft', 'Townhouse', 'Boat', 'Castle', 'Dorm',
       nan, 'Bungalow', 'Cabin', 'Tent', 'Island', 'Camper/RV', 'Plane',
       'Yurt', 'Treehouse', 'Chalet', 'Hut', 'Earth House', 'Lighthouse',
       'Parking Space', 'Tipi', 'Train'], dtype=object)